In [11]:
import imageio
import numpy as np
import matplotlib.pyplot as plt

In [25]:
class ISODATA:
    def __init__(self, image_path, starting_number_of_clusters=20, desired_number_of_clusters=10,
                 maximum_number_of_clusters=50, minimum_number_of_pixels_per_cluster=50,
                 exclusion_distance=200, closeness_criterion=30, elongation_criterion=16,
                 maximum_number_of_iterations=35, maximum_number_of_clusters_that_can_be_merged_at_one_time=2,
                 relative_decline_in_inter_cluster_center_distance=1,
                 absolute_value_of_inter_cluster_center_distance=5):
        # Read the image into a NumPy array
        image = imageio.imread(image_path)

        # Convert the image to a 2D array of pixels
        self.data = image.reshape((-1, 1))
        
        # Initialize the array to store the cluster assignments
        self.closest = np.zeros(self.data.shape[0], dtype=int)
        
        # Initialize the centroids attribute
        self.centroids = None
        
        # Store the parameters as attributes
        self.starting_number_of_clusters = starting_number_of_clusters
        self.desired_number_of_clusters = desired_number_of_clusters
        self.maximum_number_of_clusters = maximum_number_of_clusters
        self.minimum_number_of_pixels_per_cluster = minimum_number_of_pixels_per_cluster
        self.exclusion_distance = exclusion_distance
        self.closeness_criterion = closeness_criterion
        self.elongation_criterion = elongation_criterion
        self.maximum_number_of_iterations = maximum_number_of_iterations
        self.maximum_number_of_clusters_that_can_be_merged_at_one_time = maximum_number_of_clusters_that_can_be_merged_at_one_time
        self.relative_decline_in_inter_cluster_center_distance = relative_decline_in_inter_cluster_center_distance
        self.absolute_value_of_inter_cluster_center_distance = absolute_value_of_inter_cluster_center_distance

    def initialize_centroids(self):
        """returns k centroids from the initial points"""
        centroids = self.data.copy()
        np.random.shuffle(centroids)
        self.centroids = centroids[:self.starting_number_of_clusters]

    def closest_centroid(self):
        """returns an array containing the index to the nearest centroid for each point"""
        distances = np.sqrt(((self.data - self.centroids[:, np.newaxis])**2).sum(axis=2))
        self.closest = np.argmin(distances, axis=0)

    def move_centroids(self):
        """returns the new centroids assigned from the points closest to them"""
        self.centroids = np.array([self.data[self.closest==k].mean(axis=0) for k in range(self.centroids.shape[0])])
    
    def split_clusters(self):
        """Split clusters if the standard deviation along any dimension is greater than the user-defined split threshold"""
        new_centroids = []
        for k in range(self.centroids.shape[0]):
            cluster_data = self.data[self.closest==k]
            if cluster_data.shape[0] > 0:
                std_devs = np.std(cluster_data, axis=0)
                if np.any(std_devs > self.elongation_criterion):
                    new_centroids.append(self.centroids[k] - std_devs)
                    new_centroids.append(self.centroids[k] + std_devs)
                else:
                    new_centroids.append(self.centroids[k])
        self.centroids = np.array(new_centroids)

    def merge_clusters(self):
        """Merge clusters if their separation distance in multispectral feature space is less than a user-specified value"""
        new_centroids = self.centroids.copy()
        all_centroids= self.centroids.copy()
        num_merged = 0
        for i in range(self.centroids.shape[0]):
            for j in range(i+1, self.centroids.shape[0]):
                dist = np.sqrt(np.sum((self.centroids[i] - self.centroids[j])**2))
                if dist < self.closeness_criterion:
                    new_centroid = (self.centroids[i] + self.centroids[j]) / 2
                    new_centroids[i] = new_centroid
                    new_centroids = np.delete(all_centroids, j, axis=0)
                    num_merged += 1
                    if num_merged >= self.maximum_number_of_clusters_that_can_be_merged_at_one_time:
                        break
            if num_merged >= self.maximum_number_of_clusters_that_can_be_merged_at_one_time:
                break
        self.centroids = new_centroids

    def delete_clusters(self):
        """Delete clusters if they have fewer members than a user-specified minimum"""
        new_centroids = []
        for k in range(self.centroids.shape[0]):
            cluster_size = np.sum(self.closest == k)
            if cluster_size >= self.minimum_number_of_pixels_per_cluster:
                new_centroids.append(self.centroids[k])
        self.centroids = np.array(new_centroids)

    def run(self):
        """Run the ISODATA algorithm"""
        # Initialize the centroids
        self.initialize_centroids()
        
        for i in range(self.maximum_number_of_iterations):
            # Assign each data point to the closest centroid
            self.closest_centroid()
            
            # Move the centroids to the mean of the points assigned to them
            old_centroids = self.centroids.copy()
            self.move_centroids()
            
            # Check for convergence
            if np.allclose(old_centroids, self.centroids):
                print(f"Converged after {i+1} iterations")
                break
            
            # Split clusters if necessary
            self.split_clusters()
            
            # Merge clusters if necessary
            self.merge_clusters()
            
            # Delete clusters if necessary
            self.delete_clusters()
            
            print(f"Iteration {i+1}: {self.centroids.shape[0]} clusters")

In [26]:
image = imageio.imread("D:/Fourth_Year/Second_Term/BD/Flooding-Detection/dataset/flooded/0.jpg")

C:\Users\rorod\AppData\Local\Temp\ipykernel_10008\344635288.py:1: DeprecationWarning: Starting with ImageIO v3 the behavior of this function will switch to that of iio.v3.imread. To keep the current behavior (and make this warning disappear) use `import imageio.v2 as imageio` or call `imageio.v2.imread` directly.
  image = imageio.imread("D:/Fourth_Year/Second_Term/BD/Flooding-Detection/dataset/flooded/0.jpg")


In [27]:
onj=ISODATA("D:/Fourth_Year/Second_Term/BD/Flooding-Detection/dataset/flooded/0.jpg")
onj.run()

C:\Users\rorod\AppData\Local\Temp\ipykernel_10008\4001447959.py:9: DeprecationWarning: Starting with ImageIO v3 the behavior of this function will switch to that of iio.v3.imread. To keep the current behavior (and make this warning disappear) use `import imageio.v2 as imageio` or call `imageio.v2.imread` directly.
  image = imageio.imread(image_path)
C:\Users\rorod\AppData\Local\Temp\ipykernel_10008\4001447959.py:46: RuntimeWarning: Mean of empty slice.
  self.centroids = np.array([self.data[self.closest==k].mean(axis=0) for k in range(self.centroids.shape[0])])
c:\Users\rorod\miniforge3\envs\new_python_nlp\lib\site-packages\numpy\core\_methods.py:182: RuntimeWarning: invalid value encountered in divide
  ret = um.true_divide(


Iteration 1: 15 clusters
Iteration 2: 12 clusters
Iteration 3: 11 clusters
Iteration 4: 10 clusters
Iteration 5: 9 clusters
Iteration 6: 8 clusters
Iteration 7: 7 clusters
Iteration 8: 6 clusters
Iteration 9: 6 clusters
Iteration 10: 6 clusters
Iteration 11: 6 clusters
Iteration 12: 6 clusters
Iteration 13: 6 clusters
Iteration 14: 6 clusters
Iteration 15: 6 clusters
Iteration 16: 6 clusters
Iteration 17: 6 clusters
Iteration 18: 6 clusters
Iteration 19: 6 clusters
Iteration 20: 6 clusters
Iteration 21: 6 clusters
Iteration 22: 6 clusters
Converged after 23 iterations


In [35]:
# result= onj.closest.reshape((image.shape[0], image.shape[1]))

In [31]:
result.shape

(1503, 1509, 3)

In [32]:

# Create a new image array filled with zeros
mask = np.zeros_like(image)

# Assign colors to each cluster
colors = plt.cm.jet(np.linspace(0, 1, onj.centroids.shape[0]))

# Fill the mask array with colors corresponding to the cluster assignments
for k in range(onj.centroids.shape[0]):
    mask[result == k] = colors[k]

# Display the original image and the mask
plt.imshow(image)
plt.imshow(mask, alpha=0.5)
plt.show()


ValueError: NumPy boolean array indexing assignment cannot assign 4 input values to the 929032 output values where the mask is true